In [1]:
import sys
sys.path.append("..")

from src.pipeline import InsuranceQAPipeline
from src.embeddings import EmbeddingsManager
from pprint import pprint

print("Initializing pipeline..\n")
pipeline= InsuranceQAPipeline()

status=pipeline.get_status()
print("Current status:")
print(status)


Initializing pipeline..

Current status:
{'is_setup': False, 'total_documents': 0, 'LLM_provider': 'groq', 'LLM_model': 'llama-3.1-8b-instant', 'Vector_storage': {'error': 'Collection not initialized'}, 'supported_locations': 47, 'supported_procedures': 9}


In [2]:
setup_result=pipeline.setup(
    document_path="../data/raw/insurance_policy.txt",
    reset=True,
    save_chunks=True
)
print("\n Setup results:")
pprint(setup_result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


 Setup results:
{'document_path': '../data/raw/insurance_policy.txt',
 'setup_time_seconds': 0.4125077724456787,
 'statistics': {'avg_chunk_length': 368.9230769230769,
                'max_chunk_length': 446,
                'min_chunk_length': 162,
                'sections_covered': ['General',
                                     'CLAIM SUBMISSION REQUIREMENTS',
                                     'EXCLUSIONS AND NON',
                                     'SURGICAL COVERAGE AND BENEFITS',
                                     'GEOGRAPHIC COVERAGE AND NETWORK '
                                     'HOSPITALS',
                                     'AGE',
                                     'WAITING PERIODS AND IMMEDIATE COVERAGE',
                                     'CLAIM LIMITS AND FINANCIAL CAPS'],
                'total_chunks': 13,
                'unique_sections': 8},
 'success': True,
 'total_chunks': 13,
 'total_documents_stored': 13,
 'vector_db_path': 'policy_documents'}

In [3]:
import json

In [4]:
print("\n" + "="*80)
print("TEST CASE 1: Standard Approved Case")
print("="*80)

query1 = "46 year old male, knee surgery in Pune, 3 month policy"
result1 = pipeline.process_query(query1, top_k=3, verbose=True)

print("\n📄 FULL RESULT:")
print(json.dumps(result1, indent=2))


TEST CASE 1: Standard Approved Case


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📄 FULL RESULT:
{
  "query": "46 year old male, knee surgery in Pune, 3 month policy",
  "parsed_query": {
    "age": 46,
    "gender": "male",
    "procedure": "knee surgery",
    "location": "Pune",
    "policy_duration_months": 3,
    "is_emergency": false
  },
  "validation": {
    "is_complete": true,
    "missing_fields": []
  },
  "retrieved_clauses": [
    {
      "text": "2.2 CARDIAC PROCEDURES\nAngioplasty, bypass surgery, and valve replacement covered at 90% of \nactual costs. Minimum policy duration: 6 months for elective procedures.\nEmergency cardiac procedures covered immediately.\n\n2.3 TYPICAL APPROVED AMOUNTS\nBased on network hospital rates:\n- Knee arthroscopy: \u20b980,000 - \u20b91,20,000\n- Knee replacement: \u20b91,50,000 - \u20b92,50,000\n- Hip replacement: \u20b92,00,000 - \u20b93,00,000\n- Cardiac angioplasty: \u20b91,50,000 - \u20b92,00,000",
      "section": "SURGICAL COVERAGE AND BENEFITS",
      "similarity": -0.006236672401428223,
      "chunk_id": "chun

In [5]:
print("\n" + "="*80)
print("TEST CASE 2: Rejection - Insufficient Policy Duration")
print("="*80)

query2 = "45M knee surgery Mumbai 1 month policy"
result2 = pipeline.process_query(query2, top_k=3, verbose=True)

print("\n📊 Summary:")
print(f"Decision: {'✅ APPROVED' if result2['decision']['approved'] else '❌ REJECTED'}")
print(f"Reasoning: {result2['decision']['reasoning']}")



TEST CASE 2: Rejection - Insufficient Policy Duration


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📊 Summary:
Decision: ❌ REJECTED
Reasoning: The claim is rejected due to insufficient policy duration for elective surgeries. According to Section 2.1 ORTHOPEDIC PROCEDURES, elective surgeries require a minimum of 2 months policy duration, but the policy duration is only 45 months, which is not sufficient. Additionally, the patient's age (45) is within the standard coverage range, and the procedure (knee surgery) is covered at 80% of actual hospital bills. However, the emergency case status is 'No', which means the waiting period requirements apply.


In [6]:
print("\n" + "="*80)
print("TEST CASE 3: Emergency Case - Immediate Coverage")
print("="*80)

query3 = "28M emergency cardiac surgery Delhi 1 month policy"
result3 = pipeline.process_query(query3, top_k=3, verbose=True)

print("\n📊 Summary:")
print(f"Decision: {'✅ APPROVED' if result3['decision']['approved'] else '❌ REJECTED'}")
print(f"Emergency: {result3['parsed_query']['is_emergency']}")



TEST CASE 3: Emergency Case - Immediate Coverage


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📊 Summary:
Decision: ✅ APPROVED
Emergency: True


In [7]:
print("\n" + "="*80)
print("TEST CASE 4: Query with Typos")
print("="*80)

query4 = "35F hip surgry Mumbay 6 month"  # surgry, Mumbay typos
result4 = pipeline.process_query(query4, top_k=3, verbose=True)

print("\n📊 Parsed Results:")
print(f"Procedure: {result4['parsed_query']['procedure']}")
print(f"Location: {result4['parsed_query']['location']}")



TEST CASE 4: Query with Typos


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📊 Parsed Results:
Procedure: hip replacement
Location: None


In [8]:
print("\n" + "="*80)
print("TEST CASE 5: Age Boundary Case")
print("="*80)

query5 = "55 year old male knee surgery Bangalore 6 month policy"
result5 = pipeline.process_query(query5, top_k=3, verbose=True)

print("\n⚠️  Risk Factors:")
for risk in result5['decision']['risk_factors']:
    print(f"  - {risk}")


TEST CASE 5: Age Boundary Case


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


⚠️  Risk Factors:
  - Age outside standard eligibility range (18-50)
  - Potential exclusions or special conditions due to age
  - Insufficient information on emergency case status


In [9]:
print("\n" + "="*80)
print("BATCH PROCESSING TEST")
print("="*80)

batch_queries = [
    "46M knee surgery Pune 3 month policy",
    "35F hip replacement Mumbai 8 month policy",
    "60M cardiac surgery Bangalore 1 month policy",
    "28F emergency appendix surgery Delhi",
    "42F cataract surgery Chennai 12 month policy",
    "50M hernia repair Hyderabad 5 month policy"
]

batch_results = pipeline.batch_process(
    batch_queries,
    save_results=True,
    output_path="data/processed/batch_test_results.json"
)



BATCH PROCESSING TEST


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
print("\n" + "="*80)
print("BATCH RESULTS ANALYSIS")
print("="*80 + "\n")

approved_count = sum(1 for r in batch_results if r.get('decision', {}).get('approved', False))
rejected_count = len(batch_results) - approved_count

print(f"Total Queries: {len(batch_results)}")
print(f"Approved: {approved_count} ({approved_count/len(batch_results)*100:.1f}%)")
print(f"Rejected: {rejected_count} ({rejected_count/len(batch_results)*100:.1f}%)")

avg_time = sum(r.get('processing_time_seconds', 0) for r in batch_results) / len(batch_results)
print(f"\nAverage Processing Time: {avg_time:.3f} seconds")

print("\nResults by Query:")
for i, (query, result) in enumerate(zip(batch_queries, batch_results), 1):
    decision = result.get('decision', {})
    status = '✅' if decision.get('approved') else '❌'
    print(f"{i}. {status} {query[:50]}...")
    print(f"   Confidence: {decision.get('confidence', 'N/A')}")


BATCH RESULTS ANALYSIS

Total Queries: 6
Approved: 1 (16.7%)
Rejected: 5 (83.3%)

Average Processing Time: 1.982 seconds

Results by Query:
1. ❌ 46M knee surgery Pune 3 month policy...
   Confidence: low
2. ❌ 35F hip replacement Mumbai 8 month policy...
   Confidence: high
3. ❌ 60M cardiac surgery Bangalore 1 month policy...
   Confidence: low
4. ✅ 28F emergency appendix surgery Delhi...
   Confidence: high
5. ❌ 42F cataract surgery Chennai 12 month policy...
   Confidence: high
6. ❌ 50M hernia repair Hyderabad 5 month policy...
   Confidence: low


In [11]:
# Cell 10: Performance Metrics
print("\n" + "="*80)
print("PERFORMANCE METRICS SUMMARY")
print("="*80 + "\n")

all_results = [result1, result2, result3, result4, result5] + batch_results

# Processing times
times = [r['processing_time_seconds'] for r in all_results if 'processing_time_seconds' in r]
print(f"Processing Time Statistics:")
print(f"  Min: {min(times):.3f}s")
print(f"  Max: {max(times):.3f}s")
print(f"  Average: {sum(times)/len(times):.3f}s")
print(f"  Throughput: {1/(sum(times)/len(times)):.2f} queries/second")

# Accuracy metrics
complete_queries = sum(1 for r in all_results if r.get('validation', {}).get('is_complete', False))
print(f"\nQuery Parsing:")
print(f"  Complete extractions: {complete_queries}/{len(all_results)} ({complete_queries/len(all_results)*100:.1f}%)")

# Confidence distribution
confidence_dist = {}
for r in all_results:
    conf = r.get('decision', {}).get('confidence', 'unknown')
    confidence_dist[conf] = confidence_dist.get(conf, 0) + 1

print(f"\nDecision Confidence Distribution:")
for conf, count in confidence_dist.items():
    print(f"  {conf}: {count} ({count/len(all_results)*100:.1f}%)")


PERFORMANCE METRICS SUMMARY

Processing Time Statistics:
  Min: 0.588s
  Max: 8.177s
  Average: 1.430s
  Throughput: 0.70 queries/second

Query Parsing:
  Complete extractions: 9/11 (81.8%)

Decision Confidence Distribution:
  medium: 3 (27.3%)
  high: 4 (36.4%)
  low: 4 (36.4%)


In [12]:
# Cell 11: Save All Test Results
all_test_results = {
    'test_cases': {
        'case_1_standard_approved': result1,
        'case_2_rejection': result2,
        'case_3_emergency': result3,
        'case_4_typos': result4,
        'case_5_age_boundary': result5
    },
    'batch_results': batch_results,
    'performance_metrics': {
        'total_queries': len(all_results),
        'avg_processing_time': sum(times)/len(times),
        'throughput_qps': 1/(sum(times)/len(times)),
        'complete_parsing_rate': complete_queries/len(all_results),
        'confidence_distribution': confidence_dist
    }
}

with open('../data/processed/comprehensive_test_results.json', 'w') as f:
    json.dump(all_test_results, f, indent=2)

print("\n✅ All test results saved to data/processed/comprehensive_test_results.json")


✅ All test results saved to data/processed/comprehensive_test_results.json
